In [1]:
# add to path
import sys
if '../' not in sys.path:
    sys.path.insert(0, '../')
sys.path

['../',
 '/home/jimmyyao/repos/rev-stable-diffusion-2/notebooks',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python39.zip',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/lib-dynload',
 '',
 '/home/jimmyyao/.local/lib/python3.9/site-packages',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages',
 '/home/jimmyyao/repos/segment-anything',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/certifi-2022.12.7-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/wheel-0.40.0-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/setuptools-67.6.1-py3.9.egg',
 '/home/jimmyyao/repos/GenerativeImage2Text']

In [2]:
from samgit.inference import prepare_model, forward

PRETRAINED_MODEL = "../notebooks/output/SAMGIT/epoch1/model.pt"
SAM_MODEL_TYPE="vit_h"
SAM_CHECKPOINT="../models/sam_vit_h_4b8939.pth"
# TOKENIZER_PRETRAINED="/kaggle/input/bert-base-uncased"
model, mask_generator, tokenizer, param = prepare_model(PRETRAINED_MODEL,
                                                        sam_model_type=SAM_MODEL_TYPE,
                                                        # tokenizer_pretrained=TOKENIZER_PRETRAINED,
                                                        sam_checkpoint=SAM_CHECKPOINT
                                                       )

/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def caption_img(img_path: str) -> str:
    result = forward(model, mask_generator, tokenizer, param, img_path)
    return tokenizer.decode(result['predictions'][0].tolist(), skip_special_tokens=True)

def caption_img_batch(img_paths: list):
    result = forward(model, mask_generator, tokenizer, param, img_paths)
    captions = tokenizer.batch_decode(result['predictions'], skip_special_tokens=True)
    return captions

## Find and Load COCO Dataset

In [4]:
# get data
import os
IMGS_PATH = "../datasets/cococaps/val2014"

#json
import json
  
# Opening JSON file
with open('../datasets/cococaps/captions_val2014.json') as f:
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

In [5]:
data.keys()

dict_keys(['info', 'images', 'licenses', 'type', 'annotations'])

In [6]:
from torch.utils.data import DataLoader, Dataset

class COCOEvalDataset(Dataset):
    def __init__(self, data, db_base_path):
        self.data = data
        self.captions = {a['image_id']:a for a in data['annotations']}
        
        self.db_base_path = db_base_path
        
    def __len__(self):
        return len(self.data['images'])
    
    def __getitem__(self, idx):
        image_file = os.path.join(self.db_base_path, self.data['images'][idx]['file_name'])
        image_id = self.data['images'][idx]['id']
        cap = self.captions[data['images'][idx]['id']]['caption']
        return image_id, image_file, cap
    

cocoeval_dataset = COCOEvalDataset(data, IMGS_PATH)
cocoeval_dataloader = DataLoader(cocoeval_dataset, batch_size=1, shuffle=False, num_workers=0)

In [7]:
import torch
from torch.cuda.amp import autocast

import time

start = time.time()
with autocast(dtype=torch.float16):
    for i, (image_id, image_file, cap) in enumerate(cocoeval_dataloader):
        # print(image_id, image_file[0], cap)
        gen_caption = caption_img_batch(image_file)
        print({'image_id': image_id[0].item(), 'caption': gen_caption})
        if i > 2:
            break
end = time.time()
print(f"Time taken: {end-start}")

# 17.2s for 10 images autocast float16
# 17.5s for 10 images no autocast

{'image_id': 391895, 'caption': ['a man working in a village']}
{'image_id': 522418, 'caption': ['a woman in a school uniform writing a letter to a patient']}
{'image_id': 184613, 'caption': ['a herd of hippopotamus fighting in a field']}
{'image_id': 318219, 'caption': ['a photo of a young man playing a videogame in front of a microphone']}
Time taken: 8.632469415664673


In [9]:
from tqdm import tqdm

pbar = tqdm(cocoeval_dataloader)

# fake_val2014 = {'image_id':-1, "caption":"fake caption"}

with autocast(dtype=torch.float16):
    captions_val2014_fakecap_results = []
    for image_id, image_file, cap in pbar:
        pbar.set_description(f"[step {pbar.n + 1}|{image_id[0].item()}] {cap[0]}\t")
        
        gen_caption = caption_img(image_file[0])
        captions_val2014_fakecap_results.append({'image_id': image_id[0].item(), 'caption': gen_caption})
        if (pbar.n + 1) % 10000 == 0:
            with open(f'../datasets/cococaps/captions_val2014_fakecap_results{pbar.n + 1}.json', 'w') as f:
                json.dump(captions_val2014_fakecap_results, f)

[step 76|192440] A shelf with hygiene products in a bathroom.a pedestal sink.	:   0%|          | 75/40504 [01:34<13:06:57,  1.17s/it]                                        
[step 76|192440] A shelf with hygiene products in a bathroom.          
[step 185|521634] There is an image of an outdoor area. treet signs on a pole	:   0%|          | 184/40504 [04:14<72:12:27,  6.45s/it]                                      

[step 185|521634] There is an image of an outdoor area. ]                   

[step 239|495612] a street sign attached to a post with apartment buildings in the background:37,  3.71s/it]                                                                    
[step 239|495612] a street sign attached to a post with apartment buildings in the background
[step 783|301102] One tennis racket is place on top of the other one.2%|▏         | 782/40504 [18:13<12:04:11,  1.09s/it]                                                                                
[step 783|301102] One tennis 

In [10]:
with open('../datasets/cococaps/captions_val2014_fakecap_results.json', 'w') as f:
    json.dump(captions_val2014_fakecap_results, f)

In [2]:
label_file = '../datasets/cococaps/captions_val2014.json'
res_file = '../datasets/cococaps/captions_val2014_fakecap_results.json'

from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

In [3]:
coco = COCO(label_file)
cocoRes = coco.loadRes(res_file)
cocoEval = COCOEvalCap(coco, cocoRes)

cocoEval.evaluate()

result = cocoEval.eval

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 2492309 tokens at 1861383.11 tokens per second.
PTBTokenizer tokenized 589241 tokens at 2111128.06 tokens per second.


setting up scorers...
Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
computing Bleu score...
{'testlen': 346462, 'reflen': 371574, 'guess': [346462, 305958, 265454, 224952], 'correct': [191280, 77554, 26418, 9348]}
ratio: 0.9324172304843694
Bleu_1: 0.513
Bleu_2: 0.348
Bleu_3: 0.224
Bleu_4: 0.144
computing METEOR score...
METEOR: 0.160
computing Rouge score...
ROUGE_L: 0.395
computing CIDEr score...
CIDEr: 0.440
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.7 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4 sec].
Loading classif

CalledProcessError: Command '['java', '-jar', '-Xmx8G', 'spice-1.0.jar', '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/pycocoevalcap/spice/tmp/tmpz6wxu1c5', '-cache', '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/pycocoevalcap/spice/cache', '-out', '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/pycocoevalcap/spice/tmp/tmp60wvp1vc', '-subset', '-silent']' died with <Signals.SIGKILL: 9>.

In [ ]:
print(result)

with open('samgit_results.json', 'w') as fp:
    json.dump(result, fp, indent=4)